# Deep Dive: What is analyse_freenrg doing?

analyse_freenrg is a Python program that is performing a default analysis of the energy differences and gradients that are contained in the freenrgs.s3 file. You can do this analysis youself if you want to have more control, or want to get a deeper insight into the calculation.

To do this, we need to import the `Sire.Stream` module, which reads `.s3` files, and also pandas and MatplotLib so that we can draw some graphs :-)

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'   # helps make things look better in Jupyter :-)

import Sire.Stream

`Sire.Stream.read` will read all of the Python objects that are contained in a `.s3` file. The `freenrgs.s3` file contains three objects:

* bennetts - the object containing all of the data needed for a Bennetts calculation of the free energy
* fep - the object containing all of the data needed for a FEP calculation of the free energy
* ti - the object containing all of the data needed for a TI calculation of the free energy

We can load this into the notebook by using

In [ ]:
(bennetts, fep, ti) = Sire.Stream.load("output/freenrgs.s3")

All of the objects come with their own documentation, e.g.

In [ ]:
help(bennetts)

Each object contains the data collected from each iteration. We can access that data using subscripts, e.g. to get the Bennetts data from the first iteration we use

In [ ]:
data = bennetts[1]
print(data)

We can get the individual free energy averages from each λ window by inspecting this object. For example, Bennetts involves collecting both a forwards and backwards ratio for each window. We can look at the individual forwards ratios for each window using

In [ ]:
print(data.forwardsData())

All of these individual forwards and backwards ratios across all λ windows can be summed together to obtain a potential of mean force (the free energy as a function of λ). This is generated using the `sum` function

In [ ]:
pmf = bennetts[1].sum()
print(pmf)

We can plot this PMF using Matplotlib by using the below code...

In [ ]:
def plotPMF(pmf):
    x = [point.x() for point in pmf.values()]
    y = [point.y() for point in pmf.values()]
    d = DataFrame( index=x, data={"free energy":y} )
    d.plot()
    
plotPMF(pmf)

This is a pretty poor PMF as it is calculated only from the data collected from the first iteration of the calculation. This ligandswap calculation involved 1000 iterations. We can use all of the data by merging it together. We do this using the `merge` function.

In [ ]:
merged = bennetts.merge(1,1000)

The `merged` object has all of the data from all 1000 iterations merged together into a single set of averages for each λ window. We can use this to generate a PMF and calculate the free energy

In [ ]:
pmf = merged.sum()
print(pmf)
plotPMF(pmf)

Like all simulations, there is a period of equilibration that we should discard. Typically we set this as the first 40% of the calculation, everything before iteration 400. To use only the data from the last 60% of the calculation we change what we merge, e.g.

In [ ]:
merged = bennetts.merge(400,1000)
pmf = merged.sum()
print(pmf)
plotPMF(pmf)

How can we know that we are right to discard the first 40% of iterations? The best way to check is to plot the free energy predicted from each iteration and plot this. We can do this using the below code

In [ ]:
def plotConvergence(data):
    x = [i for i in range(1,1001)]
    b = [data[i].sum().values()[-1].y() for i in range(1,1001)]
    d = DataFrame( index=x, data={"free energy":b} )
    d.plot()
    
plotConvergence(bennetts)

In this case you can see that the free energy gently falls quickly at the start, and then falls gently between iterations 200 and 400, and then oscillates around an equilibrium value from iteration 400 to 1000 (although there is quite a spike after iteration 800). This gives us confidence that iterations before 400 should be discarded as equilibration, and the free energy calculated using the data from iterations 400 to 1000.

We can use similar analysis for the FEP and TI data. The only difference is that because TI uses energy gradients, it uses `integrate` instead of `sum`.

In [ ]:
pmf = ti.merge(400,1000).integrate()
print(pmf)
plotPMF(pmf)

In [ ]:
pmf = fep.merge(400,1000).sum()
print(pmf)
plotPMF(pmf)

So, there are lots of possible free energies that can come out of a single ligandswap calculation. How do you know which one is "right"? In general, I look for agreement between all of the methods. Bennetts method converges the quickest, and is generally the most reliable, while FEP is the slowest and least reliable. I look for when Bennetts, FEP and TI all agree to within 1.0 to 1.5 kcal mol-1. If they don't agree, then I run more iterations to encourage convergence, or if this takes too long, will instead use the Bennetts result. If the simulation doesn't converge, then I will plot the convergence of the different methods versus iteration, and will look for iterations where the free energy jumps away from the average. I will then look at the structures sampled at those iterations to see if a rare event has taken place (e.g. conformational change of a residue or displacement of a water molecule). If it has, then this either means that more sampling is needed, or that ligandswap is not capable of making a good prediction for this system.

If you really don't want to look at any of this, and just want a single number, then a linear or weighted average of FEP, TI and Bennetts is acceptable, e.g. `0.5 * bennetts + 0.3 * TI + 0.2 * FEP` (with those numbers based on my personal relative feeling of how much I trust each method). The error should be the spread between the different methods, e.g.

In [ ]:
b = bennetts.merge(400,1000).sum().values()[-1].y()
f = fep.merge(400,1000).sum().values()[-1].y()
t = ti.merge(400,1000).integrate().values()[-1].y()

average = 0.5*b + 0.3*f + 0.2*t
error = 0.5 * (max(f,max(b,t))-min(f,min(b,t)))

print("Bennetts = %s, FEP = %s, TI = %s" % (b,f,t))
print("Result is %s +/- %s kcal mol-1" % (average,error))

So, for the above, I would round to 1 decimal place and report the result as 4.7 +/- 0.2 kcal mol-1 (I round errors up!)